# 2021 Day 12

https://adventofcode.com/2021/day/12

In [1]:
from collections import defaultdict
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def inp_to_connections(inp):
    connections = defaultdict(list)
    for line in inp.strip().split():
        node1, node2 = line.split('-')
        connections[node1].append(node2)
        connections[node2].append(node1)
    return {**connections}

In [3]:
inp = open('input-12.txt').read()
connections = inp_to_connections(inp)
connections

{'pn': ['TY', 'co', 'rp', 'aw', 'IV', 'start'],
 'TY': ['pn', 'aw', 'co', 'ka'],
 'rp': ['ka', 'end', 'pn', 'al', 'TM'],
 'ka': ['rp', 'TY', 'IV'],
 'az': ['aw'],
 'aw': ['az', 'TY', 'pn', 'IV', 'PD'],
 'al': ['IV', 'rp', 'end'],
 'IV': ['al', 'co', 'aw', 'pn', 'ka', 'start'],
 'co': ['pn', 'IV', 'TY', 'start'],
 'end': ['rp', 'al', 'TM'],
 'TM': ['end', 'rp'],
 'PD': ['aw'],
 'start': ['IV', 'co', 'pn']}

In [4]:
test_connections = inp_to_connections("""
start-A
start-b
A-c
A-b
b-d
A-end
b-end
""")
test_connections

{'start': ['A', 'b'],
 'A': ['start', 'c', 'b', 'end'],
 'b': ['start', 'A', 'd', 'end'],
 'c': ['A'],
 'd': ['b'],
 'end': ['A', 'b']}

In [5]:
test_connections_2 = inp_to_connections("""
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
""")
test_connections_2

{'dc': ['end', 'start', 'HN', 'LN', 'kj'],
 'end': ['dc', 'HN'],
 'HN': ['start', 'dc', 'end', 'kj'],
 'start': ['HN', 'kj', 'dc'],
 'kj': ['start', 'sa', 'HN', 'dc'],
 'LN': ['dc'],
 'sa': ['kj']}

In [6]:
test_connections_3 = inp_to_connections("""
fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW
""")
test_connections_3

{'fs': ['end', 'he', 'DX', 'pj'],
 'end': ['fs', 'zg'],
 'he': ['DX', 'fs', 'pj', 'RW', 'WI', 'zg'],
 'DX': ['he', 'start', 'pj', 'fs'],
 'start': ['DX', 'pj', 'RW'],
 'pj': ['DX', 'zg', 'he', 'RW', 'start', 'fs'],
 'zg': ['end', 'sl', 'pj', 'RW', 'he'],
 'sl': ['zg'],
 'RW': ['he', 'pj', 'zg', 'start'],
 'WI': ['he']}

## Part 1

In [7]:
def traverse(connections, cur, path=None):
    if path is None:
        path = [cur]
    else:
        path = path + [cur]
    if cur == 'end':
        return [path]
    paths = []
    for conn in connections[cur]:
        if conn.islower() and conn in path:
            continue
        for fullpath in traverse(connections, conn, path=path):
            paths.append(fullpath)
    return paths

In [8]:
test_paths = traverse(test_connections, 'start')
print(len(test_paths))
test_paths

10


[['start', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'end'],
 ['start', 'A', 'end'],
 ['start', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'b', 'A', 'end'],
 ['start', 'b', 'end']]

In [9]:
test_paths_2 = traverse(test_connections_2, 'start')
print(len(test_paths_2))
test_paths_2

19


[['start', 'HN', 'dc', 'end'],
 ['start', 'HN', 'dc', 'HN', 'end'],
 ['start', 'HN', 'dc', 'HN', 'kj', 'HN', 'end'],
 ['start', 'HN', 'dc', 'kj', 'HN', 'end'],
 ['start', 'HN', 'end'],
 ['start', 'HN', 'kj', 'HN', 'dc', 'end'],
 ['start', 'HN', 'kj', 'HN', 'dc', 'HN', 'end'],
 ['start', 'HN', 'kj', 'HN', 'end'],
 ['start', 'HN', 'kj', 'dc', 'end'],
 ['start', 'HN', 'kj', 'dc', 'HN', 'end'],
 ['start', 'kj', 'HN', 'dc', 'end'],
 ['start', 'kj', 'HN', 'dc', 'HN', 'end'],
 ['start', 'kj', 'HN', 'end'],
 ['start', 'kj', 'dc', 'end'],
 ['start', 'kj', 'dc', 'HN', 'end'],
 ['start', 'dc', 'end'],
 ['start', 'dc', 'HN', 'end'],
 ['start', 'dc', 'HN', 'kj', 'HN', 'end'],
 ['start', 'dc', 'kj', 'HN', 'end']]

In [10]:
test_paths_3 = traverse(test_connections_3, 'start')
len(test_paths_3)

226

### Ok, now the actual solution

In [11]:
paths = traverse(connections, 'start')
len(paths)

4413

## Part 2

In [12]:
def traverse2(connections, cur='start', path=None):
    if path is None:
        path = [cur]
    else:
        path = path + [cur]
    if cur == 'end':
        return [path]
    paths = []
    for conn in connections[cur]:
        if conn == 'start':
            continue
        elif conn.islower() and conn in path:
            if any(path.count(c) == 2 for c in path if c.islower()):
                continue
        for fullpath in traverse2(connections, conn, path=path):
            paths.append(fullpath)
    return paths

In [13]:
test_paths = traverse2(test_connections)
print(len(test_paths))
sorted(test_paths)

36


[['start', 'A', 'b', 'A', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'b', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'd', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'd', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'd', 'b', 'end'],
 ['start', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'd', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'd', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 

In [14]:
len(traverse2(test_connections_2))

103

In [15]:
len(traverse2(test_connections_3))

3509

In [16]:
len(traverse2(connections))

118803